<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/AlBerto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AlBERTo Hate Speech Classifier 

In [1]:
!pip install ekphrasis
!pip uninstall tensorflow==2.4.1 --yes

!pip install tensorflow==1.15.0

import datetime
import sys
import warnings
warnings.filterwarnings("ignore")

#for code working
import tensorflow as tf
import re
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import numpy as np
import pandas as pd 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

#Prepare and import BERT modules
import subprocess
subprocess.call(["git", "clone", "https://github.com/google-research/bert","bert_repo"])

if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from run_classifier import *
import modeling
import optimization
import tokenization

     |████████████████████████████████| 81kB 3.2MB/s 
     |████████████████████████████████| 184kB 7.1MB/s 
     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=a81799f094d20c97cabbd64f563556f50918250184bad2516f7b9b73a926375d
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-5.9-cp37-none-any.whl size=46451 sha256=239e66cbe4f6f96f612751c61e17a645309f6750d7816e6b8fc3930165e6b832
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
Successfully built ekphrasis ftfy
Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 412.3MB 35kB/s 
     |████████████████████████████████| 512kB 30.2MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 3.8MB 

In [2]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


## Load the dataset 

In [3]:
# directory name 
input_dir = '/content/drive/My Drive/HLT/clean_dataset_training/' 
AlBERTo_path = '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/'

# Spec
pd.set_option("display.max_colwidth", None)

In [4]:
tsv_file = open(input_dir+"training_dataset.csv")

dataset = pd.read_csv(tsv_file,sep=',')
dataset = dataset[:10]

In [5]:
import pprint

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())
  

BaseException: ignored

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset['text'], dataset['hs'], test_size=0.2)


# copy of test set 
test_set = X_test  

## Configure AlBERTo classes 

In [7]:
text_processor = TextPreProcessor (
    # terms that will be normalized
    normalize=[ 'url' , 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'] ,
    # terms that will be annotated
    annotate={"hashtag"} ,
    fix_html=True ,  # fix HTML tokens

    unpack_hashtags=True ,  # perform word segmentation on hashtags

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts = [ emoticons ]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt
Reading english - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_2grams.txt
Reading english - 1grams ...


# Prepare sentences to be converted 

In [8]:
i = 0 
examples_train = []
examples_test = []

for s in X_train: 
  examples_train.append([Y_train.iloc[i], s])
  i = i+1

i = 0 
for s in X_test: 
  examples_test.append([Y_test.iloc[i], s])
  i = i+1

examples_train = np.array(examples_train)
examples_test = np.array(examples_test)

In [9]:
'''
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create InputExample's using the constructor provided in the BERT library.

    text_a is the text we want to classify, which in this case, is the Request field in our Dataframe.
    text_b is used if we're training a model to understand the relationship between sentences (i.e. is text_b a translation of text_a? Is text_b an answer to the question asked by text_a?). This doesn't apply to our task, so we can leave text_b blank.
    label is the label for our example, i.e. True, False

'''
# guid = Globally unique ID for bookkeeping, unused in this example
f = lambda x: InputExample(guid=None, text_a = x[1], text_b = None, label = int(x[0]))

examples_train = map(f, examples_train)
examples_train = list(examples_train)

examples_test = map(f, examples_test)
examples_test = list(examples_test)

# Tokenizer 

In [10]:
VOCAB_FILE = '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/vocab.txt'

#Inizialize the tokenizer
tokenizer = tokenization.FullTokenizer(VOCAB_FILE, do_lower_case=True)
# tokenizer.tokenize("Che bella giornata oggi! Stiamo procedendo bene o no? <url>")

# Load Pre-Trained Model 

In [11]:
#model = AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")

In [12]:
OUTPUT_DIR = AlBERTo_path + 'output'
#Inizialize AlBERTo
INIT_CHECKPOINT = AlBERTo_path + 'alberto_model.ckpt'
#SET THE PARAMETERS
PREDICT_BATCH_SIZE = 8
MAX_SEQ_LENGTH = 128
LEARNING_RATE = 2e-5
label_list = [0, 1]
BERT_CONFIG= modeling.BertConfig.from_json_file(AlBERTo_path + "config.json")

#SET THE PARAMETERS FOR TRAINING 
TRAIN_BATCH_SIZE = 128
PREDICT_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 512
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
MAX_SEQ_LENGTH = 64
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500
 

In [13]:
#inizialize parameters
num_train_steps = 1  # int(len(X_train) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)+
num_warmup_steps = int(NUM_TRAIN_EPOCHS * WARMUP_PROPORTION) 
print(num_train_steps)
print(num_warmup_steps)

1
0


# Create model specification 

In [14]:
model_fn = model_fn_builder(
  bert_config=BERT_CONFIG,
  num_labels=2,
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=True
)

## Setup run configuration 

In [15]:
# Setup TPU related config
#tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config():  
    return tf.contrib.tpu.RunConfig(
    #cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    #tpu_config=tf.contrib.tpu.TPUConfig(
    #    iterations_per_loop=ITERATIONS_PER_LOOP,
    #    num_shards=NUM_TPU_CORES,
    #   per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2)
    )


In [16]:
estimator = tf.contrib.tpu.TPUEstimator(
  use_tpu=False,
  model_fn=model_fn,
  predict_batch_size=PREDICT_BATCH_SIZE,
  train_batch_size=TRAIN_BATCH_SIZE,
  config=get_run_config()
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': No

# Fine-Tuning 

In [17]:
train_features = convert_examples_to_features(
      examples_train, label_list, MAX_SEQ_LENGTH, tokenizer)



INFO:tensorflow:Writing example 0 of 8
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] corriere tangenti mafia capitale dimenticata ##ma ##zzet ##te su buche e campi rom rom a [SEP]
INFO:tensorflow:input_ids: 2 726 8784 1140 1094 4196 1306 76283 521 69 9535 13 3509 2386 2386 14 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] e terrorismo anche questo per mettere in uno stato di soggezione le persone e render ##le innocue mentre qualcuno [SEP]
INFO:tensorflow:input_ids: 

In [ ]:
print("***** Running training *****")
print("  Num examples = %d", len(examples_train))
print("  Num labels = %d", len(label_list))
print("  Batch size = %d", TRAIN_BATCH_SIZE)
print("  Num steps = %d", num_train_steps)

train_input_fn = input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

print('***** Started training at {} *****'.format(datetime.datetime.now()))
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('***** Finished training at {} *****'.format(datetime.datetime.now()))

***** Running training *****
  Num examples = %d 8
  Num labels = %d 2
  Batch size = %d 128
  Num steps = %d 1
***** Started training at 2021-03-08 18:02:53.601718 *****
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (128, 64)
INFO:tensorflow:  name = input_mask, shape = (128, 64)
INFO:tensorflow:  name = label_ids, shape = (128,)
INFO:tensorflow:  name = segment_ids, shape = (128, 64)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


INFO:tensorflow:**** Traina

# Prepare predictions

In [ ]:
# Setup sentences
input_features = convert_examples_to_features(
      examples_test, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=input_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
predictions = estimator.predict(predict_input_fn)


#print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
#for example, prediction in zip(sentences,predictions):
 #   print('\t prediction:%s \t text_a: %s' % ( np.argmax(prediction['probabilities']),str(example) ) )

In [ ]:
for example, prediction in zip(test_set,predictions):
    print('\t prediction:%s \t text_a: %s' % ( prediction['probabilities'],str(example) ) )

In [ ]:
Y_test